In [1]:
import json
import re
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/shiridi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
f=open("train-v2.0.json")
js=json.load(f)

In [3]:
titles=js["data"]

In [4]:
import pandas as pd
df=pd.DataFrame()
table=[]
passage_table={}
curr_id=0
for title in titles:
    row=[]
    tit=title["title"]
    for passages in title["paragraphs"]:
        context=passages["context"]
        passage_table.update({tit+"_"+str(curr_id):context})
        passage_id=tit+"_"+str(curr_id)
        curr_id+=1
        for qas in passages["qas"]:
            question=qas["question"]
            impossible=qas["is_impossible"]
            if(not(impossible)):
                answer_start=qas["answers"][0]["answer_start"]
                answer_text=qas["answers"][0]["text"]
                _id=qas["id"]
                table.append([title["title"],passage_id,_id,question,answer_text,answer_start])
                # break
        # break
    # break


In [5]:
df=pd.DataFrame(table)

In [6]:
cols=["title","passage_id","qn_id","question","answer_text","answer_start"]
df.columns=cols


In [7]:
df

,title,passage_id,qn_id,question,answer_text,answer_start
0,Beyoncé,Beyoncé_0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,in the late 1990s,269
1,Beyoncé,Beyoncé_0,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,singing and dancing,207
2,Beyoncé,Beyoncé_0,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,2003,526
3,Beyoncé,Beyoncé_0,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,"Houston, Texas",166
4,Beyoncé,Beyoncé_0,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,late 1990s,276
...,...,...,...,...,...,...
86816,Kathmandu,Kathmandu_18979,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,Oregon,229
86817,Kathmandu,Kathmandu_18979,5735d259012e2f140011a09e,What was Yangon previously known as?,Rangoon,414
86818,Kathmandu,Kathmandu_18979,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,Minsk,476
86819,Kathmandu,Kathmandu_18979,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,1975,199


In [9]:
qn_ids={}
for i in range(len(table)):
    qn_ids.update({table[i][2]:(table[i][1],table[i][4])})


In [69]:

global ind
ind=0
def getCorrectSetence(qn):
    global ind
    pas_id,ans_text=qn_ids[qn]
    sent=sent_tokenize(passage_table[pas_id])
    flag=0
    for i in range(len(sent)):
        if(ans_text in sent[i]):
            flag=1
            return i
    
    ind+=1
    return -1


df.set_index(df["qn_id"],inplace=True)


In [70]:
df["CorrectSentence"]=df["qn_id"].apply(lambda x: getCorrectSetence(x))

In [72]:
df=df[df.CorrectSentence!=-1]

In [73]:
df

,title,passage_id,qn_id,question,answer_text,answer_start,CorrectSentence
qn_id,,,,,,,
56be85543aeaaa14008c9063,Beyoncé,Beyoncé_0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,in the late 1990s,269,1
56be85543aeaaa14008c9065,Beyoncé,Beyoncé_0,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,singing and dancing,207,1
56be85543aeaaa14008c9066,Beyoncé,Beyoncé_0,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,2003,526,3
56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé_0,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,"Houston, Texas",166,1
56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé_0,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,late 1990s,276,1
...,...,...,...,...,...,...,...
5735d259012e2f140011a09d,Kathmandu,Kathmandu_18979,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,Oregon,229,1
5735d259012e2f140011a09e,Kathmandu,Kathmandu_18979,5735d259012e2f140011a09e,What was Yangon previously known as?,Rangoon,414,2
5735d259012e2f140011a09f,Kathmandu,Kathmandu_18979,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,Minsk,476,2


In [74]:
df.to_csv("Squad_PreProcessed.csv")

In [77]:
pass_tab=[]
for  i in passage_table:
    pass_tab.append([i,passage_table[i]])
pass_df=pd.DataFrame(pass_tab)
pass_df.columns=["id","passage"]

In [79]:
pass_df.set_index("id",inplace=True)

In [87]:
pass_df

,passage
id,
Beyoncé_0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
Beyoncé_1,Following the disbandment of Destiny's Child i...
Beyoncé_2,"A self-described ""modern-day feminist"", Beyonc..."
Beyoncé_3,"Beyoncé Giselle Knowles was born in Houston, T..."
Beyoncé_4,Beyoncé attended St. Mary's Elementary School ...
...,...
Matter_19030,Isaac Newton (1643–1727) inherited Descartes' ...
Matter_19031,"There is an entire literature concerning the ""..."
Matter_19032,In the late 19th century with the discovery of...


In [89]:
pass_df.to_csv("passage_ids.csv")